# 01 Numpy

- Numpy vectorize 이해
- 다양한 연산에 친숙해지기
- row 연산 이해

<br>  
<hr>


### 1. Numpy Vectorize (Broadcasting)

파이썬의 math를 써서 sigmoid 함수를 표현해 보자.

$sigmoid(x) = \frac{1}{1+e^{-x}}$ 


In [6]:
import math

def basic_sigmoid(x):
    s = 1/(1 + math.exp(-x))
    return s 

print(basic_sigmoid(2))
#print(basic_sigmoid([1,2,3])) # TypeError: bad operand type for unary -: 'list'

0.8807970779778823


<p>math를 활용한 연산은 단일 변수에 대해 적용되므로, 벡터화 연산이 불가능하다.</p>
<p>넘파이 어레이와 넘파이 연산을 이용하면 각 개별 요소에 한번에 적용하는 병렬 연산이 가능하다.</p>

In [7]:
import numpy as np
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

t_x = np.array([1, 2, 3])
print(sigmoid(t_x))


[0.73105858 0.88079708 0.95257413]


In [8]:
# Gradients 계산
def sigmoid_derivative(x):
    s = sigmoid(x)
    ds = s*(1-s)
    return ds
print(sigmoid_derivative(t_x))

[0.19661193 0.10499359 0.04517666]


### 2. Reshaping arrays

<p>넘파이 어레이의 차원을 조작해보자.</p>
<p>가로, 세로, RGB로 구성된 3차원 이미지 어레이를 Flatten 해보자.</p>

In [10]:
def image2vector(image):
    v = image.reshape(image.shape[0]* image.shape[1]* image.shape[2], 1)    
    return v

In [11]:
t_image = np.array([[[ 0.67826139,  0.29380381],
                     [ 0.90714982,  0.52835647],
                     [ 0.4215251 ,  0.45017551]],

                   [[ 0.92814219,  0.96677647],
                    [ 0.85304703,  0.52351845],
                    [ 0.19981397,  0.27417313]],

                   [[ 0.60659855,  0.00533165],
                    [ 0.10820313,  0.49978937],
                    [ 0.34144279,  0.94630077]]])
print((image2vector(t_image)))

[[0.67826139]
 [0.29380381]
 [0.90714982]
 [0.52835647]
 [0.4215251 ]
 [0.45017551]
 [0.92814219]
 [0.96677647]
 [0.85304703]
 [0.52351845]
 [0.19981397]
 [0.27417313]
 [0.60659855]
 [0.00533165]
 [0.10820313]
 [0.49978937]
 [0.34144279]
 [0.94630077]]


### 3. Normalizing rows

성능 및 학습 속도를 높이기위해 normalize를 한다. 각 feature에 대해 진행하므로, row 기준 연산이다.

- l1 norm 절댓값
- l2 norm 유클리드 거리 
(ord 매개 변수로 지정가능)

$$x = \begin{bmatrix}
        0 & 3 & 4 \\
        2 & 6 & 4 \\
\end{bmatrix}$$ 

$$\| x\| = \text{np.linalg.norm(x, axis=1, keepdims=True)} = \begin{bmatrix}
    5 \\
    \sqrt{56} \\
\end{bmatrix} $$
$$ x\_normalized = \frac{x}{\| x\|} = \begin{bmatrix}
    0 & \frac{3}{5} & \frac{4}{5} \\
    \frac{2}{\sqrt{56}} & \frac{6}{\sqrt{56}} & \frac{4}{\sqrt{56}} \\
\end{bmatrix}$$ 

In [20]:
def normalize_rows(x):
    x_norm = np.linalg.norm(x, ord=2,axis=1, keepdims=True)
    x = x/x_norm
    return x

# np.linalg.norm을 직접 구현하면 아래와 같다.
def L1(yhat, y):
    loss = np.sum(abs(y-yhat))
    return loss
def L2(yhat, y):
    loss = np.sum(np.dot(y-yhat,y-yhat))
    return loss

In [16]:
x = np.array([[0, 3, 4],
              [2, 6, 4]])
print(normalize_rows(x))

[[0.         0.6        0.8       ]
 [0.26726124 0.80178373 0.53452248]]


### 4. Softmax
  
분류 문제에서 input의 class를 결정할 때 주로 softmax를 사용한다.

적용되는 로직은 normalize와 동일한데, 각 행 기준으로 구한 __특정값__ 을 개별 원소에 적용할 수 있다. <br>
여기서 특정값이 Broadcasting되서 적용되는 것으로 볼 수 있다.


\begin{align*}
softmax(x) &= softmax\begin{bmatrix}
            x_{11} & x_{12} & x_{13} & \dots  & x_{1n} \\
            x_{21} & x_{22} & x_{23} & \dots  & x_{2n} \\
            \vdots & \vdots & \vdots & \ddots & \vdots \\
            x_{m1} & x_{m2} & x_{m3} & \dots  & x_{mn}
            \end{bmatrix} \\ \\&= 
 \begin{bmatrix}
    \frac{e^{x_{11}}}{\sum_{j}e^{x_{1j}}} & \frac{e^{x_{12}}}{\sum_{j}e^{x_{1j}}} & \frac{e^{x_{13}}}{\sum_{j}e^{x_{1j}}} & \dots  & \frac{e^{x_{1n}}}{\sum_{j}e^{x_{1j}}} \\
    \frac{e^{x_{21}}}{\sum_{j}e^{x_{2j}}} & \frac{e^{x_{22}}}{\sum_{j}e^{x_{2j}}} & \frac{e^{x_{23}}}{\sum_{j}e^{x_{2j}}} & \dots  & \frac{e^{x_{2n}}}{\sum_{j}e^{x_{2j}}} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    \frac{e^{x_{m1}}}{\sum_{j}e^{x_{mj}}} & \frac{e^{x_{m2}}}{\sum_{j}e^{x_{mj}}} & \frac{e^{x_{m3}}}{\sum_{j}e^{x_{mj}}} & \dots  & \frac{e^{x_{mn}}}{\sum_{j}e^{x_{mj}}}
\end{bmatrix} \\ \\ &= \begin{pmatrix}
    softmax\text{(first row of x)}  \\
    softmax\text{(second row of x)} \\
    \vdots  \\
    softmax\text{(last row of x)} \\
\end{pmatrix} 
\end{align*}


In [19]:
def softmax(x):
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis = 1, keepdims=True)    
    s = x_exp/x_sum
    return s

t_x = np.array([[9, 2, 5, 0, 0],
                [7, 5, 0, 0 ,0]])
print(softmax(t_x))

[[9.80897665e-01 8.94462891e-04 1.79657674e-02 1.21052389e-04
  1.21052389e-04]
 [8.78679856e-01 1.18916387e-01 8.01252314e-04 8.01252314e-04
  8.01252314e-04]]


### 5. 다양한 Vectorization 연산



In [23]:
x1 = [9, 2, 5, 0, 0, 7, 5, 0, 0, 0, 9, 2, 5, 0, 0]
x2 = [9, 2, 2, 9, 0, 9, 2, 5, 0, 0, 9, 2, 5, 0, 0]
W = np.random.rand(3,len(x1)) # Random 3*len(x1) numpy array

dot1 = np.dot(x1,x2)
outer = np.outer(x1,x2)
mul = np.multiply(x1,x2)
dot2 = np.dot(W,x1)

print(dot1)
print(outer)
print(mul)
print(dot2)

278
[[81 18 18 81  0 81 18 45  0  0 81 18 45  0  0]
 [18  4  4 18  0 18  4 10  0  0 18  4 10  0  0]
 [45 10 10 45  0 45 10 25  0  0 45 10 25  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [63 14 14 63  0 63 14 35  0  0 63 14 35  0  0]
 [45 10 10 45  0 45 10 25  0  0 45 10 25  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [81 18 18 81  0 81 18 45  0  0 81 18 45  0  0]
 [18  4  4 18  0 18  4 10  0  0 18  4 10  0  0]
 [45 10 10 45  0 45 10 25  0  0 45 10 25  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[81  4 10  0  0 63 10  0  0  0 81  4 25  0  0]
[27.48451844 25.2827744  17.77669753]
